In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, train_test_split

import requests

# Task 1: Import the dataset

In [42]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-Coursera/medical_insurance_dataset.csv"
local_path = "./medical_insurance_dataset.csv"

request = requests.get(url)

if(request.status_code == 200):
    with open(local_path, "wb") as file:
        file.write(request.content)
    print("Success download")
else:
    print(f"Error, status code {request.status_code}")

path = "medical_insurance_dataset.csv"

Success download


<b>Gender:</b>  1 = Female; 2 = Male

<b>B.M.I.</b> = Body Mass Index of the insured

<b>No_of_Children</b> = Number of children the insured person has.

<b>Smoker: </b> 1 = Smoker; 2 = Non smoker

<b>Region: </b> 1 = Northwest; 2 = Northeast; 3 = Southwest; 4 = Southeast

<b>Charges: </b> = Charges of the insurance in USD

In [43]:
df = pd.read_csv(path, header=None)
headers = ["age", "gender", "bmi", "no_of_children", "smoker", "region", "charges"]
df.columns = headers
df

,age,gender,bmi,no_of_children,smoker,region,charges
0,19,1,27.900,0,1,3,16884.92400
1,18,2,33.770,1,0,4,1725.55230
2,28,2,33.000,3,0,4,4449.46200
3,33,2,22.705,0,0,1,21984.47061
4,32,2,28.880,0,0,1,3866.85520
...,...,...,...,...,...,...,...
2767,47,1,45.320,1,0,4,8569.86180
2768,21,1,34.600,0,0,3,2020.17700
2769,19,2,26.030,1,1,1,16450.89470
2770,23,2,18.715,0,0,1,21595.38229


In [44]:
print(df.applymap(lambda x:x == "?").sum())
df.replace("?", np.nan, inplace=True)   

age               4
gender            0
bmi               0
no_of_children    0
smoker            7
region            0
charges           0
dtype: int64


C:\Users\matia\AppData\Local\Temp\ipykernel_19292\782283507.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(df.applymap(lambda x:x == "?").sum())


# Task 2: Data Wrangling

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2772 entries, 0 to 2771
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             2768 non-null   object 
 1   gender          2772 non-null   int64  
 2   bmi             2772 non-null   float64
 3   no_of_children  2772 non-null   int64  
 4   smoker          2765 non-null   object 
 5   region          2772 non-null   int64  
 6   charges         2772 non-null   float64
dtypes: float64(2), int64(3), object(2)
memory usage: 151.7+ KB


Handle missing data:

- For continuous attributes (e.g., age), replace missing values with the mean.
- For categorical attributes (e.g., smoker), replace missing values with the most frequent value.
- Update the data types of the respective columns.

In [51]:
is_smoke = df['smoker'].value_counts().idxmax()
is_smoke

'0'